## Домашнее задание № 13. Pandas. Продвинутый 2.

### Задание 1

- [x] Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [4]:
import pandas as pd

In [21]:
df = pd.read_csv('DATA/ratings.csv')
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [22]:
# Группируем, считаем агрегации по userid
user_group_raw = df.groupby('userId').agg({'movieId':'count', 'timestamp':['min','max']}).reset_index()
user_group_raw.head()

userId movieId   timestamp            
           count         min         max
0      1      20  1260759108  1260759205
1      2      76   835355395   835356246
2      3      51  1298861589  1298932787
3      4     204   949778714   949982274
4      5     100  1163373044  1163375145

In [25]:
# фильтруем по количеству отзывов и создаем колонку со временем жизни
user_group_100_review = user_group_raw[user_group_raw['movieId']['count'] > 100].copy()
user_group_100_review['lifetime_utc'] = user_group_100_review.loc[:,('timestamp','max')] - user_group_100_review.loc[:,('timestamp','min')]
user_group_100_review.head()

userId movieId   timestamp             lifetime_utc
            count         min         max             
3       4     204   949778714   949982274       203560
7       8     116  1154389340  1154474527        85187
14     15    1700   997937239  1469330735    471393496
16     17     363  1127468587  1127476640         8053
18     19     423   855190091   855195373         5282

In [26]:
from datetime import timedelta
# получаем финальный результат и выводим в днях

finalta = round(user_group_100_review.lifetime_utc.mean() / timedelta(days=1).total_seconds(),2)
print(f'Средняя продолжительность жизни для пользователей с количеством отзывов более 100 : {finalta} дней.')

Средняя продолжительность жизни для пользователей с количеством отзывов более 100 : 463.89 дней.


### Задание 2

- [x] Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). 

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

Необходимо сформировать две таблицы:

1. таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
2. аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [5]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

In [6]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

In [7]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

In [8]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [19]:
# Создаем объедененный датафрейм через метод append и заменяем пустые значения на нули
result = rzd.append(air, ignore_index=True).append(auto, ignore_index=True).apply(lambda x: x.fillna(0))
result.head()

,client_id,rzd_revenue,air_revenue,auto_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,0.0,0.0
3,114,5774.0,0.0,0.0
4,115,981.0,0.0,0.0


In [20]:
# разворачиваем таблицу, а потом сводим
result_unpivot = result.melt(id_vars=['client_id'], var_name='source', value_name='revenue')
result_1st = result_unpivot.pivot_table(index='client_id', columns='source', values='revenue', 
                                        aggfunc='sum', fill_value=0).reset_index()
result_1st

source,client_id,air_revenue,auto_revenue,rzd_revenue
0,111,0,0,1093
1,112,0,0,2810
2,113,0,57483,10283
3,114,0,83,5774
4,115,81,912,981
5,116,4,4834,0
6,117,13,98,0
7,118,173,0,0


In [34]:
# Присоединяем к ней информацию с адресами и выводим адреса после client_id

result_2nd = result_1st.join(client_base.set_index('client_id'),on='client_id')
new_order = list(result_2nd.columns.values)
new_order[1], new_order[4] = new_order[4], new_order[1]
result_2nd = result_2nd.reindex(columns=new_order)
result_2nd

,client_id,address,auto_revenue,rzd_revenue,air_revenue
0,111,Комсомольская 4,0,1093,0
1,112,Энтузиастов 8а,0,2810,0
2,113,Левобережная 1а,57483,10283,0
3,114,Мира 14,83,5774,0
4,115,ЗЖБИиДК 1,912,981,81
5,116,Строителей 18,4834,0,4
6,117,Панфиловская 33,98,0,13
7,118,Мастеркова 4,0,0,173
